###Content Based Filtering

In [0]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [0]:
users = pd.read_csv('users.csv')
posts = pd.read_csv('posts.csv').fillna('None')
views = pd.read_csv('views.csv')
posts.head()
columns  = list(posts.columns)
columns[2] , columns[3] = columns[3] , columns[2]
posts.columns = columns


In [0]:
posts['features'] = posts['category']+ ' ' +posts[' post_type']

In [0]:

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 6), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(posts['features'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)


In [0]:
results = {}

for idx, row in posts.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]

    similar_items = [(cosine_similarities[idx][i], posts['_id'][i]) for i in similar_indices]
    results[row['_id']] = similar_items[1:]


In [0]:

def item(id):
  return (posts.loc[posts['_id'] == id]['title'].tolist()[0],posts.loc[posts['_id'] == id]['category'].tolist()[0])

def recommend(item_id, num):
    a,b=item(item_id)
    print('similar posts to {} of Category :{} '.format(a,b))
    recs = results[item_id][:num]
    for rec in recs:
        a,b = item(rec[1])
        print('\n',a,'-- CATEGORY :',b)
        


In [0]:
recommend('5ec821ddec493f4a2655889e',10)

similar posts to Save Earth. of Category :artwork 

 'The Virtual ME' -- CATEGORY : artwork

 EID MUBARAK -- CATEGORY : artwork

 Magazine Cover Redefined -- CATEGORY : artwork

 Design Process -- CATEGORY : artwork

 Gun store Logo. -- CATEGORY : artwork

 Break The Chain: Stop The Spread -- CATEGORY : artwork

 Handwashing 101 -- CATEGORY : artwork

 Be kind during Corona -- CATEGORY : artwork

 Corona's Checklist -- CATEGORY : artwork

 Is there any vaccine of COVID-19? -- CATEGORY : artwork


In [0]:
post_id = '5d6d39567fa40e1417a4931c'
recommend(post_id,10)

similar posts to Ml and AI of Category :blog 

 Machine Learning”&“Operations" (MlOps) -- CATEGORY : blog

 Learning... -- CATEGORY : blog

 GAN's INTRODUCTION -- CATEGORY : blog

 Recommend Systems Machine Learning -- CATEGORY : blog

 7 Best Python Data Science Courses & Certification [2020] -- CATEGORY : blog

 GAN's Part(2) -- CATEGORY : blog

 10 Best Artificial Intelligence (AI) Courses Online for 2020 -- CATEGORY : blog

 Artificial Intelligence -- CATEGORY : blog

 Types Of AI. -- CATEGORY : blog

 What sports will look like in the future -- CATEGORY : blog


In [0]:
post_id = '5e4c28c5f5561b1994c8e3b0'
recommend(post_id,10)

similar posts to E-Commerce - Business Models of Category :blog 

 Benefits of Buying Grocery Online in Twin city of Odisha -- CATEGORY : blog

 EDI (Electronic data interchange) -- CATEGORY : blog

 E-Cash -- CATEGORY : blog

 E-Commerce -- CATEGORY : blog

 Students Networking! -- CATEGORY : blog

 Marital Rape - Rape is Rape -- CATEGORY : blog

 Biodiversity -- CATEGORY : blog

 How Competition law evolved? -- CATEGORY : blog

 Raghavan Committee - 2002. -- CATEGORY : blog

 Let's discuss some Case laws! -- CATEGORY : blog


###Collaborative Filtering

In [0]:
views['seen'] = 1
merged1 = pd.merge(views,users,how='left' ,left_on='user_id',right_on='_id').drop('_id', axis=1)
data = pd.merge(merged1,posts,how='left' ,left_on='post_id',right_on='_id').drop('_id', axis=1)

data =data.dropna()

pivot = pd.pivot_table(data,index='user_id',columns='post_id',values='seen',fill_value=0)

correlation_posts = pivot.corr()
correlation_users = pivot.transpose().corr()


In [0]:
def get_similar_users(user_id):
    similar_users = correlation_users[user_id]
    similar_users = similar_users.sort_values(ascending=False)[1:10]
    return similar_users


In [0]:
def recommend(user,n):
    print('Posts viewed by users similar to',users.loc[users['_id']==user]['name'].tolist())
    similar_users_id = get_similar_users(user)
    reccs = []
    for id in similar_users_id.index:
        posts_recs = (data[data['user_id'] == id]['title'].tolist())
        for post in posts_recs:
            reccs.append(post)
    return reccs[:n]


In [0]:
user = '5d618359fc5fcf3bdd9a0910'

recommend(user,10)


Posts viewed by users similar to ['Akshay Mishra']


['Cloud Computing',
 'Learning...',
 'Launching an EC2 instance.',
 'AWS CLI Setup in Mac',
 'Configure Docker with Django; PostgreSQL; Pg-admin & Elasticsearch',
 'CI/CD using GitHub Actions',
 'Face Recognition using Transfer Learning.',
 'Recommendation Engine',
 'We can overcome these days!',
 'Designing Circular Microstrip Antenna']

In [0]:
user = '5e3563348d344822fed4d13a'
recommend(user,15)

Posts viewed by users similar to ['Ojasvi Arya']


['The Waves Have Stood Still',
 'Machine Learning”&“Operations" (MlOps)',
 'Faith in yourself',
 'Zero-Waste Lifestyle',
 'Novel & Consenent (P.H.P)',
 'Always do Right 👍🏻',
 'Art Expo 2020',
 'Save Earth.',
 'Kabir singh',
 'Form of ma durga',
 'HUMAN ILLUSTRATION (With Galaxy Background)',
 'HUMAN ILLUSTRATION ( Background of a room)',
 'FASHION ILLUSTRATION',
 'HUMAN ILLUSTRATION',
 'FASHION ILLUSTRATION (OP ART)']

In [0]:
user = '5e1ef04c2a37d20505da2b8b'
recommend(user,15)

Posts viewed by users similar to ['Rashi Jain']


['Moore FSM Sequence Detector',
 'Understanding Cloud Computing(AWS)',
 'Photography Composition',
 'Designing Cmos circuit from Boolean expressions (Microwind)',
 'The Waves Have Stood Still',
 'Machine Learning”&“Operations" (MlOps)',
 'Faith in yourself',
 'Beliefs ❣️',
 'Form of ma durga',
 'HUMAN ILLUSTRATION ( Background of a room)',
 'HUMAN ILLUSTRATION (With Galaxy Background)',
 'HUMAN ILLUSTRATION',
 'FASHION ILLUSTRATION',
 'FASHION ILLUSTRATION (OP ART)',
 'FASHION ILLUSTRATION (street fashion)']